# Logistic Regression

## Probit Model

### Context

A Probit Regression Model is a version of the generalized linear model used to model dichotomous outcome variables. It uses the inverse standard normal distribution as a linear combination of the predictors. The binary outcome variable Y is assumed to have a Bernoulli distribution with parameter p (where the success probability is p ∈ (0,1)). 

The probit link function is probit(EY) = f(Pr(Y=1|X))


Please note:

* Link function used for Logistic regression: n(p) = ln(p/1-p)
* Link function used for Probit regression: n(p) = f(p)

We will use the ```Mroz``` data set on ```Female Labor Participation``` with 8 variables. The data covers a sample of 750+ married women aged between 30 and 60 collected in 1975. The original source for this data is Mroz, T.A. (1987). Ref: http://unionstats.gsu.edu/9220/Mroz_Econometrica_LaborSupply_1987.pdf

Dataset Ref: https://github.com/Accelerate-AI/Classification-Models/blob/main/data/Mroz.csv 

|Variable(s)	   |Description	                                  |Type          |
|---------------|----------------------------------------------|--------------|
|lfp        	|Labor-force participation of the married white woman|Categorical: 0/1|
|k5 	        |Number of children younger than 6 years old	     |Positive integer|
|k618    	    |Number of children aged 6-18	                     |Positive integer|
|age        	|Age in years	                                     |Positive integer|
|wc	            |Wifes college attendance	                         |Categorical: 0/1|
|hc	            |Husbands college attendance	                     |Categorical: 0/1|
|lwg        	|Log expected wage rate for women in the labor force |Numerical       |
|inc 	        |Family income without the wifes income	             |Numerical       |

Our objective is to identify whether certain characteristics of a woman’s household and personal life can predict her labor-force participation.

### Import Libraries and Load Dataset

In [1]:
import pandas as pd 
import numpy as np

import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Probit

In [2]:
# Loading the dataset 
df = pd.read_csv('C:/Users/mishr/Desktop/Notebooks/data/Mroz.csv') 

df.sample(5)

,Unnamed: 0,lfp,k5,k618,age,wc,hc,lwg,inc
696,697,no,0,1,48,yes,yes,1.526319,22.299999
665,666,no,0,0,55,no,no,1.039452,13.071999
96,97,yes,0,1,45,no,no,1.580689,23.799999
415,416,yes,1,2,39,yes,no,-1.543298,16.120001
212,213,yes,0,2,41,no,no,1.357383,15.600000


Data Cleaning - Since some of the data is read in as strings, we need to transform them into binary categorical data using the following code. We also drop the first column as it is read in with row numbers, which we do not need.

In [3]:
df = df.drop(df.columns[0], axis = 1)
df["lfp"] = df["lfp"] == "yes"
df["wc"] = df["wc"] == "yes"
df["hc"] = df["hc"] == "yes"

In [4]:
df.sample(5)

,lfp,k5,k618,age,wc,hc,lwg,inc
581,False,0,2,37,True,True,1.065162,15.900000
310,True,0,1,30,False,False,1.196087,15.591000
549,False,0,1,47,False,False,1.121527,17.700001
647,False,0,0,49,False,False,0.851477,18.150000
416,True,0,2,36,True,False,1.912046,26.500000


In [5]:
df.describe()

,k5,k618,age,lwg,inc
count,753.000000,753.000000,753.000000,753.000000,753.000000
mean,0.237716,1.353254,42.537849,1.097115,20.128965
std,0.523959,1.319874,8.072574,0.587556,11.634799
min,0.000000,0.000000,30.000000,-2.054124,-0.029000
25%,0.000000,0.000000,36.000000,0.818086,13.025000
50%,0.000000,1.000000,43.000000,1.068403,17.700001
75%,0.000000,2.000000,49.000000,1.399717,24.466000
max,3.000000,8.000000,60.000000,3.218876,96.000000


### Fit the Probit Model

First we break our dataset into response variable and predictor variables. We will use lfp as our response variable and all the remaining variable as predictors. Then we use the statsmodels function to fit our Probit regression with our response variable and design matrix. The statsmodels package is unique from other languages and packages as it does not include an intercept term by default. This needs to be manually set.

In [6]:
y = df["lfp"]
X = df.drop(["lfp"], 1)
X = sm.add_constant(X)

#probit_model = Probit(y, X).fit()
probit_model = Probit(y, X.astype(float)).fit()

Optimization terminated successfully.
         Current function value: 0.601189
         Iterations 5


The following is the results of our regression. The statsmodels package automatically includes p values and confidence intervals for each coefficient. The probit model is stored as a probit model object in Python. All operations with the model are invoked as model.member_function().

In [7]:
print(probit_model.summary())

                          Probit Regression Results                           
Dep. Variable:                    lfp   No. Observations:                  753
Model:                         Probit   Df Residuals:                      745
Method:                           MLE   Df Model:                            7
Date:                Fri, 10 Jun 2022   Pseudo R-squ.:                  0.1208
Time:                        21:14:28   Log-Likelihood:                -452.69
converged:                       True   LL-Null:                       -514.87
Covariance Type:            nonrobust   LLR p-value:                 9.471e-24
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9184      0.381      5.040      0.000       1.172       2.664
k5            -0.8747      0.114     -7.703      0.000      -1.097      -0.652
k618          -0.0386      0.040     -0.953      0.3

It appears that all our predictors are statistically significant at level a except hc and k618

### Marginal Effects

To better understand why our predictors are significant, we attempt to study the marginal effects of our predictors. Here, we will try to look at the predictors hc, wc, age, and k5. Unfortunately, there is no member function of the probit class to calculate the standard error of each prediction so this is ommited here.

#### Group by Husband’s College Attendance

In [8]:
# First we group the data by hc
# To better understand hc, we tabulate lfp and hc. This is done with the pandas crosstab function.

print(pd.crosstab(df["lfp"], df["hc"], margins = True))

hc     False  True  All
lfp                    
False    207   118  325
True     251   177  428
All      458   295  753


Then we calculate adjusted predictions of ```lfp``` for two levels of the ```hc``` variable. Creating these adjusted predictors is not very clean in Python compared to other languages. We have to build the array from scratch.

In [9]:
df.columns

Index(['lfp', 'k5', 'k618', 'age', 'wc', 'hc', 'lwg', 'inc'], dtype='object')

In [10]:
hc_data0 = np.column_stack((
    1,
    np.mean(df["k5"]),
    np.mean(df["k618"]),
    np.mean(df["age"]),
    np.mean(df["wc"]),
    0,
    np.mean(df["lwg"]),
    np.mean(df["inc"])
    ))
    
hc_data1 = np.column_stack((
    1,
    np.mean(df["k5"]),
    np.mean(df["k618"]),
    np.mean(df["age"]),
    np.mean(df["wc"]),
    1,
    np.mean(df["lwg"]),
    np.mean(df["inc"])
    ))

In [11]:
print(probit_model.predict(hc_data0))
print(probit_model.predict(hc_data1))

[0.56938181]
[0.59171968]


We see that the marginal probability of husband being a collage graduate is 0.591, while the marginal probability of husband being a high school graduate is lower at 0.569.

#### Group by Wife’s College Attendance

Similarly to ```hc```, we tabulate our response variable ```lfp``` and predictor ```wc```. We can see that there is a significant imbalance between labor force participation for women that are college graduates.

In [12]:
print(pd.crosstab(df["lfp"], df["wc"], margins = True))

wc     False  True  All
lfp                    
False    257    68  325
True     284   144  428
All      541   212  753


We can also calculate the adjusted predictions of ```lfp``` for the two levels of ```wc``` variable, while keeping other variables at the mean.

In [13]:
wc_data0 = np.column_stack((
    1,
    np.mean(df["k5"]),
    np.mean(df["k618"]),
    np.mean(df["age"]),
    0,
    np.mean(df["hc"]),
    np.mean(df["lwg"]),
    np.mean(df["inc"])
    ))
wc_data1 = np.column_stack((
    1,
    np.mean(df["k5"]),
    np.mean(df["k618"]),
    np.mean(df["age"]),
    1,
    np.mean(df["hc"]),
    np.mean(df["lwg"]),
    np.mean(df["inc"])
    ))

In [14]:
print(probit_model.predict(wc_data0))
print(probit_model.predict(wc_data1))

[0.52380974]
[0.70816505]


It appears that the marginal probability increases from 0.524 to 0.708 when women are college graduates compared to highschool graduates.

#### Group by Age and Wife’s College Attendance

We can predict the outcome variable by both age and women’s college education. We use increments of 10 between 30 and 60 as our representative ages.

In [15]:
wc_data0 = np.column_stack((
    np.repeat(1,4),
    np.repeat(np.mean(df["k5"]),4),
    np.repeat(np.mean(df["k618"]), 4),
    (30,40,50,60),
    np.repeat(0,4),
    np.repeat(np.mean(df["hc"]),4),
    np.repeat(np.mean(df["lwg"]),4),
    np.repeat(np.mean(df["inc"]),4)
    ))
wc_data1 = np.column_stack((
    np.repeat(1,4),
    np.repeat(np.mean(df["k5"]),4),
    np.repeat(np.mean(df["k618"]), 4),
    (30,40,50,60),
    np.repeat(1,4),
    np.repeat(np.mean(df["hc"]),4),
    np.repeat(np.mean(df["lwg"]),4),
    np.repeat(np.mean(df["inc"]),4)
))

In [16]:
print(probit_model.predict(wc_data0))
print(probit_model.predict(wc_data1))

[0.70330952 0.5618684  0.41195181 0.27399923]
[0.84667045 0.74021954 0.6047985  0.45523423]


As we can see, the older a women gets, the less chance of her participating in the labor force. Regardless of age, women’s college education always yields a higher probability of labor force participation.

#### Group by Number of Children 5 years or younger (K5)

Now, let's finally look at the effects of number of children 5 years or younger. As before, here is a cross tabulation of this variable.

In [17]:
print(pd.crosstab(df["lfp"], df["k5"], margins = True))

k5       0    1   2  3  All
lfp                        
False  231   72  19  3  325
True   375   46   7  0  428
All    606  118  26  3  753


Then we predict ```lfp``` at different levels of ```k5```, while keeping other variables at their means.

In [18]:
k5_data = np.column_stack((
    np.repeat(1,4),
    (0,1,2,3),
    np.repeat(np.mean(df["k618"]), 4),
    np.repeat(np.mean(df["age"]),4),
    np.repeat(np.mean(df["wc"]),4),
    np.repeat(np.mean(df["hc"]),4),
    np.repeat(np.mean(df["lwg"]),4),
    np.repeat(np.mean(df["inc"]),4)
    ))

In [19]:
print(probit_model.predict(k5_data))

[0.65730924 0.31932735 0.08942703 0.01324326]


We see that if there are three or more children under the age of 5, there is a 0.013 chance of a woman being in the work force, but when there is no children there is a 0.6573 chance. This is a good indication of the significance of this variable.

#### Extension

Now, an overall marginal effect can be observed by calling the ```get_margeff()``` method of the probit model class. This is unique to Python. Results are as follows.

In [20]:
mfx = probit_model.get_margeff()
print(mfx.summary())

       Probit Marginal Effects       
Dep. Variable:                    lfp
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
k5            -0.2997      0.034     -8.726      0.000      -0.367      -0.232
k618          -0.0132      0.014     -0.955      0.340      -0.040       0.014
age           -0.0130      0.002     -5.219      0.000      -0.018      -0.008
wc             0.1673      0.045      3.696      0.000       0.079       0.256
hc             0.0196      0.042      0.461      0.645      -0.064       0.103
lwg            0.1253      0.029      4.311      0.000       0.068       0.182
inc           -0.0070      0.002     -4.451      0.000      -0.010      -0.004


### Summary

In [21]:
print(probit_model.summary())

                          Probit Regression Results                           
Dep. Variable:                    lfp   No. Observations:                  753
Model:                         Probit   Df Residuals:                      745
Method:                           MLE   Df Model:                            7
Date:                Fri, 10 Jun 2022   Pseudo R-squ.:                  0.1208
Time:                        21:14:28   Log-Likelihood:                -452.69
converged:                       True   LL-Null:                       -514.87
Covariance Type:            nonrobust   LLR p-value:                 9.471e-24
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9184      0.381      5.040      0.000       1.172       2.664
k5            -0.8747      0.114     -7.703      0.000      -1.097      -0.652
k618          -0.0386      0.040     -0.953      0.3

Hence the Regression equation is as follows:

probit(EY) = f(Pr(lfp=1)) 
           = 1.9184 - 0.8747 * k5 - 0.0386 * k618 - 0.0378 * age + 0.4883 * wc + 0.0572 * hc + 0.3656 * lwg - 0.0205 * inc

We predict the probability of labor-force participation by grouping a certain variable, while keeping other variables at mean. Grouping by ```hc```, there is no obivious difference between probabilities, which confirms that ```hc``` is not a significant indicator in our model. We do the same for ```wc```, ```age```, and ```k5```. We can conclude that these varibles have effects on labor-force participation.